In [1]:
from transformers import AutoTokenizer
import torch
from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling
from tqdm import tqdm
from hybrid_model import HybridModel
from datasets import load_dataset
from transformers import MambaModel

/u/r/i/riyad/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/u/r/i/riyad/miniconda3/lib/python3.12/site-packages/mamba_ssm/ops/selective_scan_interface.py:163: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/u/r/i/riyad/miniconda3/lib/python3.12/site-packages/mamba_ssm/ops/selective_scan_interface.py:239: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/u/r/i/riyad/miniconda3/lib/python3.12/site-packages/mamba_ssm/ops/triton/layer_norm.py:985: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @c

In [2]:
seed, buffer_size = 42, 10_000
dataset = load_dataset('HuggingFaceFW/fineweb', split='train', streaming=True)
dataset = dataset.shuffle(seed, buffer_size=buffer_size)

In [3]:
transformer_tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neo-125M',padding_side="left")
# if transformer_tokenizer.mask_token is None:
#     transformer_tokenizer.add_special_tokens({'mask_token': '[MASK]'})

In [4]:
dataset = dataset.with_format("torch")
# transformer_model = AutoModelForCausalLM.from_pretrained(
#     'EleutherAI/gpt-neo-125M',
#     torch_dtype="auto",
#     device_map="cuda"
# )
# mamba_model = MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf")

In [7]:
#transformer_model

In [5]:
from transformers import AutoModel
transformer_backbone= AutoModel.from_pretrained('EleutherAI/gpt-neo-125M')

In [5]:
transformer_backbone

GPTNeoModel(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(2048, 768)
  (drop): Dropout(p=0.0, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPTNeoBlock(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPTNeoAttention(
        (attention): GPTNeoSelfAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): Linear(in_features=768, out_features=768, bias=False)
          (v_proj): Linear(in_features=768, out_features=768, bias=False)
          (q_proj): Linear(in_features=768, out_features=768, bias=False)
          (out_proj): Linear(in_features=768, out_features=768, bias=True)
        )
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPTNeoMLP(
        (c_fc): Linear(in_features=768, out_features=3072, bias=True)
        (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        (act): NewGELUActivation()

In [6]:
mamba_backbone = MambaModel.from_pretrained('state-spaces/mamba-130m-hf')

In [10]:
mamba_backbone

MambaModel(
  (embeddings): Embedding(50280, 768)
  (layers): ModuleList(
    (0-23): 24 x MambaBlock(
      (norm): MambaRMSNorm(768, eps=1e-05)
      (mixer): MambaMixer(
        (conv1d): Conv1d(1536, 1536, kernel_size=(4,), stride=(1,), padding=(3,), groups=1536)
        (act): SiLU()
        (in_proj): Linear(in_features=768, out_features=3072, bias=False)
        (x_proj): Linear(in_features=1536, out_features=80, bias=False)
        (dt_proj): Linear(in_features=48, out_features=1536, bias=True)
        (out_proj): Linear(in_features=1536, out_features=768, bias=False)
      )
    )
  )
  (norm_f): MambaRMSNorm(768, eps=1e-05)
)

In [7]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

transformer_backbone = transformer_backbone.to(device)
mamba_backbone = mamba_backbone.to(device)


In [12]:
for param in transformer_backbone.parameters():
    param.requires_grad = False

for param in mamba_backbone.parameters():
    param.requires_grad = False
    
model = HybridModel(transformer_backbone, mamba_backbone) 

# model.to(device) ? unfreeze : not
model = model.train().to(device)

trainer_params = []
for combiner in model.combiners.parameters():
    trainer_params.append(combiner)

for spliter in model.splitters.parameters():
    trainer_params.append(spliter)
    
#trainer_params = torch.T(trainer_params)
print(trainer_params[0])
len(trainer_params)

Parameter containing:
tensor([[ 1.8396e-03, -1.3812e-02,  4.9040e-03,  ..., -9.8580e-03,
         -3.2768e-02, -2.6284e-02],
        [-9.5766e-03, -1.7582e-02,  2.4014e-02,  ..., -7.6723e-05,
          4.8130e-03, -4.6479e-03],
        [-3.1165e-02,  4.9692e-04,  2.5671e-02,  ..., -1.7864e-02,
         -2.5413e-02, -2.9034e-02],
        ...,
        [ 3.0531e-02,  1.8112e-02, -1.8730e-02,  ...,  3.3435e-02,
          9.0439e-03, -5.9737e-03],
        [-1.3401e-02, -2.4215e-02, -2.3823e-02,  ...,  3.0899e-02,
          3.4544e-02,  3.4608e-02],
        [-1.4139e-02, -1.0430e-02,  3.0011e-02,  ..., -3.3844e-02,
         -1.0488e-02, -1.8261e-02]], device='cuda:0', requires_grad=True)


96

In [13]:
mamba_backbone

MambaModel(
  (embeddings): Embedding(50280, 768)
  (layers): ModuleList(
    (0-23): 24 x MambaBlock(
      (norm): MambaRMSNorm(768, eps=1e-05)
      (mixer): MambaMixer(
        (conv1d): Conv1d(1536, 1536, kernel_size=(4,), stride=(1,), padding=(3,), groups=1536)
        (act): SiLU()
        (in_proj): Linear(in_features=768, out_features=3072, bias=False)
        (x_proj): Linear(in_features=1536, out_features=80, bias=False)
        (dt_proj): Linear(in_features=48, out_features=1536, bias=True)
        (out_proj): Linear(in_features=1536, out_features=768, bias=False)
      )
    )
  )
  (norm_f): MambaRMSNorm(768, eps=1e-05)
)

In [8]:
if transformer_tokenizer.pad_token is None:
    transformer_tokenizer.pad_token = transformer_tokenizer.unk_token if transformer_tokenizer.unk_token else transformer_tokenizer.eos_token

In [15]:
batched_dataset = dataset.batch(batch_size=16)

In [16]:
for batch in batched_dataset:
    t = transformer_tokenizer(batch["text"], padding="max_length", truncation=True, return_tensors= "pt")
    print(t , batch)
    break

/u/r/i/riyad/miniconda3/lib/python3.12/site-packages/datasets/formatting/torch_formatter.py:78: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(value, **{**default_dtype, **self.torch_tensor_kwargs})


{'input_ids': tensor([[50256, 50256, 50256,  ...,   285,  2621, 19679],
        [50256, 50256, 50256,  ...,  1708,  6050,    30],
        [50256, 50256, 50256,  ...,   286, 11278,    13],
        ...,
        [50256, 50256, 50256,  ...,    13,  1157, 22199],
        [50256, 50256, 50256,  ...,    13,   447,   251],
        [50256, 50256, 50256,  ...,   329,  3555,     0]]), 'attention_mask': tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        ...,
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]])} {'text': ['If you go winter backpacking, snowshoeing, or mountaineering, you are going to want to bring at least one pair of mittens. Some people will even bring two pairs in case one gets wet. Personally, I bring a pair of mittens for warmth and a pair of gloves for added dexterity.\nWhen selecting mittens and gloves for winter conditions, here are a couple of must-have features tha

In [12]:
def tokenize_function(examples):
    return transformer_tokenizer(examples["text"],truncation=True)
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['text', 'id', 'dump', 'url', 'date', 'file_path', 'language', 'language_score', 'token_count'])
# Create the data collator for language modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=transformer_tokenizer, mlm=False)

# Create the dataloader
dataloader = DataLoader(tokenized_dataset, batch_size=8, collate_fn=data_collator)

In [61]:
for token in tokenized_dataset:
    print(token['input_ids'])
    print(token.keys())
    break

tensor([ 1532,   345,   467,  7374,   736, 41291,    11,  6729,  1477,  2577,
          278,    11,   393,  3817,  5718,  1586,    11,   345,   389,  1016,
          284,   765,   284,  2222,   379,  1551,   530,  5166,   286,   285,
        34978,    13,  2773,   661,   481,   772,  2222,   734, 14729,   287,
         1339,   530,  3011,  9583,    13, 31342,    11,   314,  2222,   257,
         5166,   286,   285, 34978,   329, 23125,   290,   257,  5166,   286,
        18051,   329,  2087, 50003,    13,   198,  2215, 17246,   285, 34978,
          290, 18051,   329,  7374,  3403,    11,   994,   389,   257,  3155,
          286,  1276,    12, 14150,  3033,   326,   345,   815,   804,   329,
           25,   198,  8413, 21985,  9493,   364,    25,   921,   761,   257,
          285,  2621,   393, 29144,  1080,   326,   468,  9493,   364,   290,
          257, 35973,  7582,   523,   345,   836,   447,   247,    83,   625,
        25080,   290, 15488,    13, 19372,   265,   287,  7374, 

/u/r/i/riyad/miniconda3/lib/python3.12/site-packages/datasets/formatting/torch_formatter.py:78: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(value, **{**default_dtype, **self.torch_tensor_kwargs})


In [71]:
# from projector import Combiner
# c = Combiner(10,10).to(device)

# c.in_proj1.weight.device

# cm = torch.nn.ModuleList([Combiner(10, 10) for _ in range(12)]).to(device)




In [10]:

device = 'cuda' if torch.cuda.is_available() else 'cpu' 

for param in transformer_backbone.parameters():
    param.requires_grad = False

for param in mamba_backbone.parameters():
    param.requires_grad = False
    
model = HybridModel(transformer_backbone, mamba_backbone,device) 

# model.to(device) ? unfreeze : not
model.train()

trainer_params = []
for combiner in model.combiners.parameters():
    trainer_params.append(combiner)

for spliter in model.splitters.parameters():
    trainer_params.append(spliter)
optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-5)
for e in range(3):
    # shuffle the dataset
    dataset.set_epoch(e)
    for i, batch in enumerate(dataloader):
        if i == 5:
            break
        #batch = {k: v.to(device) for k, v in batch.items() if isinstance(v, torch.Tensor)}
        outputs = model(input_data = batch['input_ids'])
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if i % 10 == 0:
            print(f"loss: {loss}")

WEIGHT cuda:0


/u/r/i/riyad/miniconda3/lib/python3.12/site-packages/datasets/formatting/torch_formatter.py:78: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(value, **{**default_dtype, **self.torch_tensor_kwargs})


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [44]:
dataset.__getitem__

<bound method Dataset.__getitem__ of IterableDataset({
    features: ['text', 'id', 'dump', 'url', 'date', 'file_path', 'language', 'language_score', 'token_count'],
    n_shards: 23781
})>

In [ ]:
#dataloader = DataLoader(tokenized_dataset,batch_size= 32, collate_fn=DataCollatorForLanguageModeling(transformer_tokenizer,mlm=False))




  0%|          | 0/5 [00:00<?, ?it/s]/u/r/i/riyad/miniconda3/lib/python3.12/site-packages/datasets/formatting/torch_formatter.py:78: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(value, **{**default_dtype, **self.torch_tensor_kwargs})
  0%|          | 0/5 [00:02<?, ?it/s]

{'language_score': tensor([0.9335, 0.9738, 0.9514, 0.9592, 0.8248, 0.9719, 0.8839, 0.9658, 0.9717,
        0.8748, 0.9487, 0.9108, 0.9771, 0.8554, 0.9519, 0.9758],
       device='cuda:0'), 'token_count': tensor([539, 391, 119, 658,  71, 192, 352, 391, 796, 513, 109, 104, 294, 207,
        194, 629], device='cuda:0')}


RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.FloatTensor instead (while checking arguments for embedding)

In [65]:
model.splitters[0].out_proj1.weight.device

device(type='cuda', index=0)

In [66]:
model.transformer_model.device

device(type='cuda', index=0)

In [69]:
t['input_ids'].to(device)

tensor([[50256, 50256, 50256,  ...,   285,  2621, 19679],
        [50256, 50256, 50256,  ...,  1708,  6050,    30],
        [50256, 50256, 50256,  ...,   286, 11278,    13],
        ...,
        [50256, 50256, 50256,  ...,    13,  1157, 22199],
        [50256, 50256, 50256,  ...,    13,   447,   251],
        [50256, 50256, 50256,  ...,   329,  3555,     0]], device='cuda:0')

In [35]:

optimizer = torch.optim.AdamW(params=trainer_params , lr=1e-5)
for epoch in range(3):
    dataset.set_epoch(epoch)
    for i, batch in enumerate(tqdm(batched_dataset, total=5)):
        if i == 5:
            break

        #batch = {k: v.to(device) for k, v in batch.items() if isinstance(v, torch.Tensor)}
        print(batch)
        t = transformer_tokenizer(batch["text"], padding="max_length", truncation=True, return_tensors= "pt")
        outputs = transformer_backbone(t['input_ids'])
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if i % 10 == 0:
            print(f"loss: {loss}")

  0%|          | 0/5 [00:00<?, ?it/s]/u/r/i/riyad/miniconda3/lib/python3.12/site-packages/datasets/formatting/torch_formatter.py:78: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(value, **{**default_dtype, **self.torch_tensor_kwargs})
  0%|          | 0/5 [00:02<?, ?it/s]


{'text': ['If you go winter backpacking, snowshoeing, or mountaineering, you are going to want to bring at least one pair of mittens. Some people will even bring two pairs in case one gets wet. Personally, I bring a pair of mittens for warmth and a pair of gloves for added dexterity.\nWhen selecting mittens and gloves for winter conditions, here are a couple of must-have features that you should look for:\nRemovable liners: You need a mitten or glove system that has liners and a removable shell so you don’t overheat and sweat. Sweat in winter is nasty because it freezes in place and chills you, so you have to be extra vigilant about removing layers when you get hot and slow your activity levels to the point where you can stay dry. You can also put removable liners in your sleeping bag at night to dry them.\nWaterproof, breathable shell: When you sweat you need a shell fabric that has the ability to vent moisture and prevent it from getting in. Higher denier (tougher) Gore-Tex shells wo

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)